In [ ]:
import os
import json
import argparse
import time
import torch
from easynmt import EasyNMT

from tqdm import tqdm

In [ ]:
abbre2language = {
    "en": "english",
    "af": "afrikaans",
    "ar": "arabic",
    "bg": "bulgarian",
    "bn": "bengali",
    "de": "german",
    "el": "greek",
    "es": "spanish",
    "et": "estonian",
    "eu": "basque",
    "fa": "persian",
    "fi": "finnish",
    "fr": "french",
    "he": "hebrew",
    "hi": "hindi",
    "hu": "hungarian",
    "id": "indonesian",
    "it": "italian",
    "ja": "japanese",
    "jv": "javanese",
    "ka": "georgian",
    "kk": "kazakh",
    "ko": "korean",
    "ml": "malayalam",
    "mr": "marathi",
    "ms": "malay",
    "my": "burmese",
    "nl": "dutch",
    "pt": "portuguese",
    "ru": "russian",
    "sw": "swahili",
    "ta": "tamil",
    "te": "telugu",
    "th": "thai",
    "tl": "tagalog",
    "tr": "turkish",
    "ur": "urdu",
    "vi": "vietnamese",
    "yo": "yoruba",
    "zh": "chinese",
}

language2abbre = {v: k for k, v in abbre2language.items()}

In [ ]:
device = 'cuda:0'
translator = EasyNMT('m2m_100_418M', device=device)

In [ ]:
lang_abbres = ["es", "it", "ru", "zh", "ja"] # "de", "fr"
num_trn_examples = 10000

# mldoc_ = "/input/jongwooko/xlt/data/download/mldoc/"
mldoc_ = "./mldoc/MLDoc-main/mldoc/"

# for which_split, wsplit in (("train", "trn"), ("test", "tst"), ):
for which_split, wsplit in (("test", "tst"), ):
    for abbr in lang_abbres:
        lang = None
        lang = abbre2language[abbr]
        print (abbr, lang)
        if which_split == "train":
            which_split = f"{lang}.train.{num_trn_examples}"
        if which_split == "dev":
            which_split = f"{lang}.dev"
        if which_split == "test":
            which_split = f"{lang}.test"
        file_ = os.path.join(mldoc_, which_split)
        
        trans_line = []
        with open(file_, "r") as f:
            for idx, line in tqdm(enumerate(f)):
                line = line.strip().split("\t")
                
                if abbr != "en":
                    assert len(line) == 2
                portion_identifier = -1
                label = line[0].strip()
                text_a = line[1].strip()
                print (len(text_a))
                
                with torch.no_grad():
                    text_b = translator.translate(text_a, source_lang=abbr, target_lang='en')
                trans_line.append(label + "\t" + text_a + "\t" + text_b.strip())
                
                if idx == 10:
                    break
        
        trans_file_ = file_.replace(lang, '{}_english'.format(lang))
        print (trans_file_)
        
        with open(trans_file_, "w") as trans_f:
            for i in trans_line:
                trans_f.write(i + "\n")